In [ ]:
!pip install polars
!pip install fugashi[unidic]
!pip install --upgrade --no-cache-dir gdown

In [12]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
!gdown "https://drive.google.com/uc?export=download&id=1L2ENYi0vxpwT0HhHElamCb6mxG3VpFv6"
!unzip ai_lab_2023-20230128T005645Z-001

In [ ]:
prefix_files = "/content/ai_lab_2023"

# 演習
## LDAモデルを使って、類似文書の検索
1. livedoor newsコーパスのカテゴリーごとに８：２の割合でデータセットを学習用・テスト用に分割してください
1. トピック数を2~10の中で、最もパープレキシティの低いLDAモデルを作ってください
1. 各カテゴリーごとのテストデータを入力として使って、LDAモデルによって最も類似度の高い文書を学習データから選定してください。
1. 上記で、選定されたデータが入力したデータと同じカテゴリーかどうかを判定してください。同じカテゴリーの場合は成功とします。
1. カテゴリーごとに成功率を計算してください。

## word2vec/k-meansとLDAの比較
1. 上記で学習させたLDAの各トピックの上位10個ずつ単語を抽出してください。
1. これらの単語をword2vecで単語ベクトルに変換してください。
1. これらの単語ベクトル集合をk-meansでクラスタリングしてください。ただし、k-meansのクラスタ数はLDAのトピック数と同じにしてください。
1. 1で抽出したLDAのトピックの単語集合とk-meansのクラスタの単語集合を比較してください。

In [17]:
import re
import requests
import polars as pl
from fugashi import Tagger
from gensim.corpora import Dictionary, MmCorpus
from gensim import models
from sklearn.model_selection import train_test_split

class LivedoorCorpus():
    def __init__(self, df):
        self.df = df

        # 全角半角文字以外（記号と数字）を正規表現を使って除去
        pattern = r"[^\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\u20000-\u2ffff\sa-zA-Z]"
        self.raw_documents = [re.sub(pattern, "", text) for text in self.df["DOCUMENT"]]
        # Mecabで分かち書きして、単語に分割
        self.raw_documents = [Tagger('-Owakati').parse(text).split() for text in self.raw_documents]
        # ストップワードの除去
        self.raw_documents = self._rm_stopwords()
        # 1文字は除去
        self.raw_documents = [[word for word in text if len(word) > 1]for text in self.raw_documents]

        self.dictionary = Dictionary(self.raw_documents)

        self.bow = [ self.dictionary.doc2bow(text) for text in self.raw_documents]


    def reset_dict_corpus(self):
        self.dictionary = Dictionary(self.raw_documents)
        self.bow = [ self.dictionary.doc2bow(text) for text in self.raw_documents]

    def print_stats(self):
        print(f"文書数: {self.dictionary.num_docs}, " + f"語彙数: {len(self.dictionary)}")

    def dict_top_n(self, top_n: int):
        most_frequent_ids = (v for v in self.dictionary)
        most_frequent_ids = sorted(most_frequent_ids, key=self.dictionary.dfs.get, reverse=True)
        most_frequent_ids = most_frequent_ids[:top_n]
        return [self.dictionary[idx] for idx in most_frequent_ids]
        
    def _rm_stopwords(self):
        # ストップワードの準備
        stopwords_url = "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
        r = requests.get(stopwords_url)
        tmp = r.text.split('\r\n')
        stopwords = []
        for i in range(len(tmp)):
            if len(tmp[i]) < 1:
                continue
            stopwords.append(tmp[i])

        return [[word for word in text if not word in stopwords]for text in self.raw_documents]

2023-01-27 11:05:29,509 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-01-27 11:05:30,680 : INFO : built Dictionary<64506 unique tokens: ['10', '11', '111029', '2005', '2006']...> from 5903 documents (total 1657278 corpus positions)
2023-01-27 11:05:30,681 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<64506 unique tokens: ['10', '11', '111029', '2005', '2006']...> from 5903 documents (total 1657278 corpus positions)", 'datetime': '2023-01-27T11:05:30.681656', 'gensim': '4.3.0', 'python': '3.9.6 (default, Sep 26 2022, 11:37:49) \n[Clang 14.0.0 (clang-1400.0.29.202)]', 'platform': 'macOS-12.5.1-arm64-arm-64bit', 'event': 'created'}
2023-01-27 11:05:40,520 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-01-27 11:05:40,816 : INFO : built Dictionary<33154 unique tokens: ['11', '12', '2011', '2012', '300']...> from 1464 documents (total 413316 corpus positions)
2023-01-27 11:05:40,817 : INFO : Dictionary lifecycle event {'msg': "built 

In [22]:
import pickle

df_train = pl.read_csv(f'{prefix_files}/raw_corpus_train.csv')
df_test = pl.read_csv(f'{prefix_files}/raw_corpus_test.csv')
with open(f'{prefix_files}/train_corpus.pkl', 'rb') as f:
    train_corpus = pickle.load(f)
dictionary = Dictionary.load(f'{prefix_files}/livedoor.dict')
train_bow = MmCorpus(f'{prefix_files}/train_bow.mm')
test_bow = MmCorpus(f'{prefix_files}/test_bow.mm')

In [23]:
topic_range = range(2, 10)

def calc_perplexity(m, c):
    import numpy as np
    return np.exp(-m.log_perplexity(c))

def search_model(corpus_train, corpus_test):
    most = [1.0e6, None]
    print(f"dataset: training/test = {len(corpus_train)}/{len(corpus_test)}")

    for t in topic_range:
        # 辞書はtrain_corpusのものを使います。
        m = models.LdaModel(corpus=corpus_train, id2word=train_corpus.dictionary, num_topics=t, iterations=500, passes=5)
        # m = models.LdaModel(corpus=corpus_train, id2word=dictionary, num_topics=t, iterations=500, passes=5)
        p1 = calc_perplexity(m, corpus_train)
        p2 = calc_perplexity(m, corpus_test)
        print(f"{t}: perplexity is {p1}/{p2}")
        
        if p2 < most[0]:
            most[0] = p2
            most[1] = m
    
    return most[0], most[1]

perplexity, model_lda = search_model(train_bow, test_bow)
print(f"Best model: topics={model_lda.num_topics}, perplexity={perplexity}")

2023-01-27 11:07:07,999 : INFO : using symmetric alpha at 0.5
2023-01-27 11:07:08,000 : INFO : using symmetric eta at 0.5
2023-01-27 11:07:08,008 : INFO : using serial LDA version on this node
2023-01-27 11:07:08,013 : INFO : running online (multi-pass) LDA training, 2 topics, 5 passes over the supplied corpus of 5903 documents, updating model once every 2000 documents, evaluating perplexity every 5903 documents, iterating 500x with a convergence threshold of 0.001000
2023-01-27 11:07:08,026 : INFO : PROGRESS: pass 0, at document #2000/5903


dataset: training/test = 5903/1464


2023-01-27 11:07:15,784 : INFO : merging changes from 2000 documents into a model of 5903 documents
2023-01-27 11:07:15,838 : INFO : topic #0 (0.500): 0.008*"映画" + 0.007*"できる" + 0.004*"より" + 0.004*"公開" + 0.004*"話題" + 0.004*"日本" + 0.004*"記事" + 0.003*"まで" + 0.003*"発売" + 0.003*"対応"
2023-01-27 11:07:15,839 : INFO : topic #1 (0.500): 0.009*"映画" + 0.005*"できる" + 0.005*"写真" + 0.004*"公開" + 0.004*"より" + 0.004*"記事" + 0.004*"たい" + 0.004*"日本" + 0.004*"作品" + 0.004*"話題"
2023-01-27 11:07:15,839 : INFO : topic diff=1.574989, rho=1.000000
2023-01-27 11:07:15,854 : INFO : PROGRESS: pass 0, at document #4000/5903
2023-01-27 11:07:18,689 : INFO : merging changes from 2000 documents into a model of 5903 documents
2023-01-27 11:07:18,691 : INFO : topic #0 (0.500): 0.006*"できる" + 0.004*"日本" + 0.004*"より" + 0.004*"映画" + 0.004*"まで" + 0.003*"関連" + 0.003*"だけ" + 0.003*"話題" + 0.003*"記事" + 0.003*"発売"
2023-01-27 11:07:18,691 : INFO : topic #1 (0.500): 0.005*"日本" + 0.005*"たい" + 0.004*"記事" + 0.004*"って" + 0.004*"まで" + 0.0

2: perplexity is 3289.611828383107/3676.412447422048


2023-01-27 11:07:49,180 : INFO : merging changes from 2000 documents into a model of 5903 documents
2023-01-27 11:07:49,182 : INFO : topic #0 (0.333): 0.006*"映画" + 0.006*"できる" + 0.005*"写真" + 0.005*"作品" + 0.004*"日本" + 0.004*"より" + 0.003*"まで" + 0.003*"記事" + 0.003*"公開" + 0.003*"だろう"
2023-01-27 11:07:49,183 : INFO : topic #1 (0.333): 0.008*"映画" + 0.006*"できる" + 0.006*"話題" + 0.005*"記事" + 0.004*"より" + 0.004*"公開" + 0.004*"たい" + 0.004*"日本" + 0.004*"機能" + 0.004*"アプリ"
2023-01-27 11:07:49,183 : INFO : topic #2 (0.333): 0.010*"映画" + 0.007*"できる" + 0.005*"公開" + 0.005*"より" + 0.004*"まで" + 0.004*"作品" + 0.004*"だろう" + 0.004*"記事" + 0.004*"日本" + 0.003*"たい"
2023-01-27 11:07:49,184 : INFO : topic diff=1.756413, rho=1.000000
2023-01-27 11:07:49,184 : INFO : PROGRESS: pass 0, at document #4000/5903
2023-01-27 11:07:52,157 : INFO : merging changes from 2000 documents into a model of 5903 documents
2023-01-27 11:07:52,159 : INFO : topic #0 (0.333): 0.006*"日本" + 0.004*"って" + 0.004*"あり" + 0.004*"まで" + 0.004*"たい" + 

...

7: perplexity is 2839.793493761835/3769.846033311618


2023-01-27 11:11:01,126 : INFO : merging changes from 2000 documents into a model of 5903 documents
2023-01-27 11:11:01,130 : INFO : topic #5 (0.125): 0.008*"できる" + 0.007*"映画" + 0.005*"日本" + 0.005*"PC" + 0.004*"より" + 0.004*"スマート" + 0.004*"だろう" + 0.004*"機能" + 0.004*"公開" + 0.003*"画面"
2023-01-27 11:11:01,130 : INFO : topic #7 (0.125): 0.008*"映画" + 0.007*"できる" + 0.006*"写真" + 0.005*"たい" + 0.004*"記事" + 0.004*"より" + 0.004*"まで" + 0.004*"なく" + 0.004*"日本" + 0.004*"だけ"
2023-01-27 11:11:01,131 : INFO : topic #6 (0.125): 0.010*"写真" + 0.008*"撮影" + 0.005*"映画" + 0.005*"できる" + 0.005*"アプリ" + 0.004*"iPhone" + 0.004*"より" + 0.004*"記事" + 0.004*"たい" + 0.003*"だけ"
2023-01-27 11:11:01,131 : INFO : topic #1 (0.125): 0.011*"映画" + 0.007*"公開" + 0.005*"できる" + 0.005*"世界" + 0.005*"より" + 0.004*"まで" + 0.004*"日本" + 0.004*"作品" + 0.004*"登場" + 0.004*"だろう"
2023-01-27 11:11:01,133 : INFO : topic #2 (0.125): 0.009*"できる" + 0.006*"映画" + 0.006*"話題" + 0.005*"機能" + 0.005*"日本" + 0.005*"だろう" + 0.004*"テレビ" + 0.004*"より" + 0.004*"記事" + 

8: perplexity is 2842.8961262275834/3874.141620281034


2023-01-27 11:11:38,149 : INFO : merging changes from 2000 documents into a model of 5903 documents
2023-01-27 11:11:38,152 : INFO : topic #2 (0.111): 0.009*"できる" + 0.006*"アプリ" + 0.006*"映画" + 0.006*"日本" + 0.004*"iPhone" + 0.004*"だろう" + 0.004*"より" + 0.004*"まで" + 0.004*"だけ" + 0.004*"公開"
2023-01-27 11:11:38,152 : INFO : topic #1 (0.111): 0.017*"映画" + 0.009*"作品" + 0.008*"公開" + 0.005*"監督" + 0.005*"より" + 0.005*"日本" + 0.004*"世界" + 0.004*"まで" + 0.004*"できる" + 0.004*"本作"
2023-01-27 11:11:38,153 : INFO : topic #8 (0.111): 0.007*"社長" + 0.005*"PC" + 0.005*"より" + 0.005*"話題" + 0.005*"iPhone" + 0.004*"記事" + 0.004*"映画" + 0.004*"まで" + 0.004*"機能" + 0.004*"関連"
2023-01-27 11:11:38,154 : INFO : topic #5 (0.111): 0.018*"映画" + 0.007*"公開" + 0.005*"話題" + 0.005*"できる" + 0.005*"日本" + 0.004*"記事" + 0.004*"作品" + 0.004*"10" + 0.004*"監督" + 0.004*"登場"
2023-01-27 11:11:38,155 : INFO : topic #7 (0.111): 0.006*"映画" + 0.006*"できる" + 0.004*"日本" + 0.004*"作品" + 0.004*"iPhone" + 0.004*"より" + 0.004*"まで" + 0.003*"なく" + 0.003*"だけ" 

9: perplexity is 2787.86337955878/3913.4706381160295
Best model: topics=5, perplexity=3526.663859752393


In [24]:
import numpy as np
from gensim import similarities
index = similarities.MatrixSimilarity(model_lda[train_bow], num_features=len(train_corpus.dictionary))

acc = []
loss = []
for i, doc in enumerate(test_bow):
    similarity_lda = index[doc]
    if df_train[int(np.argmax(similarity_lda))]["CATEGORY"][0] == df_test[i]["CATEGORY"][0]:
        acc.append([df_train[int(np.argmax(similarity_lda))]["DOCUMENT"][0], df_test[i]["DOCUMENT"][0]])
    else:
        loss.append([df_train[int(np.argmax(similarity_lda))]["DOCUMENT"][0], df_test[i]["DOCUMENT"][0]])


2023-01-27 11:12:26,075 : INFO : creating matrix with 5903 documents and 11877 features


In [25]:
len(acc)/(len(acc) + len(loss))

0.08469945355191257

In [26]:
class WVCorpus():
    def __init__(self, corpus):
        self.corpus = corpus
    def __iter__(self):
        return iter(self.corpus)

sentences = WVCorpus(train_corpus.raw_documents)
# instantiating and training the Word2Vec model
model_wv = models.Word2Vec(
    sentences,
    min_count=1,
    compute_loss=True,
    hs=0,
    sg=1,
    seed=42,
)

# getting the training loss value
training_loss = model_wv.get_latest_training_loss()
print(training_loss)

2023-01-27 11:13:14,205 : INFO : collecting all words and their counts
2023-01-27 11:13:14,211 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-01-27 11:13:14,562 : INFO : collected 64506 word types from a corpus of 1657278 raw words and 5903 sentences
2023-01-27 11:13:14,563 : INFO : Creating a fresh vocabulary
2023-01-27 11:13:15,020 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 64506 unique words (100.00% of original 64506, drops 0)', 'datetime': '2023-01-27T11:13:15.020803', 'gensim': '4.3.0', 'python': '3.9.6 (default, Sep 26 2022, 11:37:49) \n[Clang 14.0.0 (clang-1400.0.29.202)]', 'platform': 'macOS-12.5.1-arm64-arm-64bit', 'event': 'prepare_vocab'}
2023-01-27 11:13:15,021 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 1657278 word corpus (100.00% of original 1657278, drops 0)', 'datetime': '2023-01-27T11:13:15.021430', 'gensim': '4.3.0', 'python': '3.9.6 (default, Sep 26 2022, 11:37:49) \n[Clang 1

25265774.0


In [27]:
from sklearn.cluster import KMeans

topic_words = []
for t in range(model_lda.num_topics):
    x = model_lda.show_topic(t, 10)
    topic_words.append([(t, w[0]) for w in x])

topic_df = pl.DataFrame({str(x[0][0]):[word[1] for word in x] for x in topic_words})
topic_words = np.ravel([[word[1] for word in x] for x in topic_words])

word_vectors = [model_wv.wv[x] for x in topic_words]

kmeans_clustering = KMeans(n_clusters = model_lda.num_topics)
idx = kmeans_clustering.fit_predict(word_vectors)

y = []
for t in range(model_lda.num_topics):
    _y = []
    for i, id in enumerate(idx):
        if t == id:
            _y.append((t, topic_words[i]))
    y.append(_y)


cluster_words = {str(w[0][0]):[word[1] for word in w] for w in y}
print(topic_df)
for k,v in cluster_words.items():
    print(k,v)

shape: (10, 5)
┌──────────┬────────┬────────────┬──────┬────────┐
│ 0        ┆ 1      ┆ 2          ┆ 3    ┆ 4      │
│ ---      ┆ ---    ┆ ---        ┆ ---  ┆ ---    │
│ str      ┆ str    ┆ str        ┆ str  ┆ str    │
╞══════════╪════════╪════════════╪══════╪════════╡
│ スマート ┆ 写真   ┆ ソフト     ┆ って ┆ 日本   │
│ フォン   ┆ アプリ ┆ バンク     ┆ 女性 ┆ 映画   │
│ SM       ┆ できる ┆ できる     ┆ いい ┆ 監督   │
│ AX       ┆ 撮影   ┆ 応募       ┆ たい ┆ ネット │
│ ...      ┆ ...    ┆ ...        ┆ ...  ┆ ...    │
│ Android  ┆ ゲーム ┆ プレゼント ┆ たら ┆ 公開   │
│ 機能     ┆ たい   ┆ まで       ┆ なく ┆ 放送   │
│ ドコモ   ┆ 表示   ┆ 人気       ┆ たり ┆ テレビ │
│ アプリ   ┆ 紹介   ┆ 関連       ┆ 男性 ┆ られ   │
└──────────┴────────┴────────────┴──────┴────────┘
0 ['できる', '記事', 'ゲーム', 'たい', '紹介', 'できる', '話題', 'より', 'プレゼント', 'まで', '人気', '関連', 'って', '女性', 'いい', 'たい', 'けど', '結婚', 'たら', 'なく', 'たり', '男性', '日本', '映画', 'ネット', '選手', '世界', '公開', 'テレビ', 'られ']
1 ['ソフト', 'バンク']
2 ['写真', '撮影', '監督', '放送']
3 ['スマート', 'フォン', 'SM', 'AX']
4 ['対応', '更新', 'Android', '機能', 'ドコモ', 'アプリ',

/Users/hajime/projects/ai_lab/.env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [28]:
topic_df

0,1,2,3,4
str,str,str,str,str
"""スマート""","""写真""","""ソフト""","""って""","""日本"""
"""フォン""","""アプリ""","""バンク""","""女性""","""映画"""
"""SM""","""できる""","""できる""","""いい""","""監督"""
"""AX""","""撮影""","""応募""","""たい""","""ネット"""
"""対応""","""画面""","""話題""","""けど""","""選手"""
"""更新""","""記事""","""より""","""結婚""","""世界"""
"""Android""","""ゲーム""","""プレゼント""","""たら""","""公開"""
"""機能""","""たい""","""まで""","""なく""","""放送"""
"""ドコモ""","""表示""","""人気""","""たり""","""テレビ"""


In [29]:
for k,v in cluster_words.items():
    print(k,v)

0 ['できる', '記事', 'ゲーム', 'たい', '紹介', 'できる', '話題', 'より', 'プレゼント', 'まで', '人気', '関連', 'って', '女性', 'いい', 'たい', 'けど', '結婚', 'たら', 'なく', 'たり', '男性', '日本', '映画', 'ネット', '選手', '世界', '公開', 'テレビ', 'られ']
1 ['ソフト', 'バンク']
2 ['写真', '撮影', '監督', '放送']
3 ['スマート', 'フォン', 'SM', 'AX']
4 ['対応', '更新', 'Android', '機能', 'ドコモ', 'アプリ', 'アプリ', '画面', '表示', '応募']
